In [ ]:
!wget https://vedas.sac.gov.in/static/pdf/SIH_2022/SS594_Multispectral_Dehazing.zip
!unzip SS594_Multispectral_Dehazing.zip
!mv SS594_Multispectral_Dehazing/Haze1k/Haze1k/Haze1k_thin/dataset/valid SS594_Multispectral_Dehazing/Haze1k/Haze1k/Haze1k_thin/dataset/val
!mv SS594_Multispectral_Dehazing/Haze1k/Haze1k/Haze1k_thick/dataset/valid SS594_Multispectral_Dehazing/Haze1k/Haze1k/Haze1k_thick/dataset/val

In [2]:
%pip install opencv-python
!apt-get update
!apt-get install -y libgl1-mesa-glx

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3373 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1637 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:6 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [23.8 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1688 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]    
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [1728 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2411 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3785 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [30

In [3]:
import cv2

def DarkChannel(im, sz):
    b, g, r = cv2.split(im)
    dc = cv2.min(cv2.min(r, g), b)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (sz, sz))
    dark = cv2.erode(dc, kernel)
    return dark

In [4]:
def AtmLight(im, dark):
    [h, w] = im.shape[:2]
    imsz = h * w
    numpx = int(max(math.floor(imsz / 1000), 1))
    darkvec = dark.reshape(imsz)
    imvec = im.reshape(imsz, 3)

    indices = darkvec.argsort()
    indices = indices[imsz - numpx : :]

    atmsum = np.zeros([1, 3])
    for ind in range(1, numpx):
        atmsum = atmsum + imvec[indices[ind]]

    A = atmsum / numpx
    return A

In [5]:
def TransmissionEstimate(im, A, sz):
    omega = 0.95
    im3 = np.empty(im.shape, im.dtype)

    for ind in range(0, 3):
        im3[:, :, ind] = im[:, :, ind] / A[0, ind]

    transmission = 1 - omega * DarkChannel(im3, sz)
    return transmission

In [6]:
def Guidedfilter(im, p, r, eps):
    mean_I = cv2.boxFilter(im, cv2.CV_64F, (r, r))
    mean_p = cv2.boxFilter(p, cv2.CV_64F, (r, r))
    mean_Ip = cv2.boxFilter(im * p, cv2.CV_64F, (r, r))
    cov_Ip = mean_Ip - mean_I * mean_p

    mean_II = cv2.boxFilter(im * im, cv2.CV_64F, (r, r))
    var_I = mean_II - mean_I * mean_I

    a = cov_Ip / (var_I + eps)
    b = mean_p - a * mean_I

    mean_a = cv2.boxFilter(a, cv2.CV_64F, (r, r))
    mean_b = cv2.boxFilter(b, cv2.CV_64F, (r, r))

    q = mean_a * im + mean_b
    return q

In [7]:
def TransmissionRefine(im, et):
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray) / 255
    r = 60
    eps = 0.0001
    t = Guidedfilter(gray, et, r, eps)

    return t

In [9]:
from enum import Enum

class DatasetType(Enum):
    Train = 0,
    Test = 1,
    Validation = 2

    def ToString(self) -> str:
        if self == DatasetType.Train:
            return 'train'
        elif self == DatasetType.Test:
            return 'test'
        elif self == DatasetType.Validation:
            return 'val'

In [10]:
import pathlib
import os
import math
import numpy as np

def GenerateTransmissionMaps(datasetType: DatasetType, verbose=False):
    dehazingDatasetPath = pathlib.Path("SS594_Multispectral_Dehazing/Haze1k/Haze1k")

    hazyImagePaths = []
    clearImagePaths = []

    hazyImages = []
    clearImages = []
    estimateMaps = []
    refinedMaps = []

    for variant in ("Haze1k_thin", "Haze1k_moderate", "Haze1k_thick"):
        inputPath = dehazingDatasetPath / variant / "dataset" / datasetType.ToString() / "input"
        targetPath = dehazingDatasetPath / variant / "dataset" / datasetType.ToString() / "target"

        hazyImagePaths += [
            inputPath / filename
            for filename in sorted(os.listdir(inputPath))
            if filename.lower().endswith((".png", ".jpg", ".jpeg", ".tiff", ".bmp"))
        ]
        clearImagePaths += [
            targetPath / filename
            for filename in sorted(os.listdir(targetPath))
            if filename.lower().endswith((".png", ".jpg", ".jpeg", ".tiff", ".bmp"))
        ]

    print(len(hazyImagePaths), len(clearImagePaths))
    # Filtering the mismatching (input, target) image pair
    assert len(hazyImagePaths) == len(clearImagePaths)
    for hazyPath, clearPath in zip(hazyImagePaths, clearImagePaths):
        hazyImage = cv2.imread(str(hazyPath))
        clearImage = cv2.imread(str(clearPath))

        if hazyImage is None or clearImage is None:
            print(hazyPath, clearPath)

        valid = hazyImage is not None and clearImage is not None and hazyImage.shape == clearImage.shape
        if not valid:
            hazyImagePaths.remove(hazyPath)
            clearImagePaths.remove(clearPath)
            continue

        if verbose:
            print(hazyPath)
            print(clearPath)

        # Generate Transmission Maps
        I = hazyImage.astype("float64") / 255

        dark = DarkChannel(I, 15)
        A = AtmLight(I, dark)
        estimate = TransmissionEstimate(I, A, 15)
        refined = TransmissionRefine(hazyImage, estimate)

        hazyImages.append(hazyImage)
        clearImages.append(clearImage)
        estimateMaps.append(estimate)
        refinedMaps.append(refined)

    print(len(hazyImages), len(clearImages), len(estimateMaps), len(refinedMaps))
    return hazyImages, clearImages, estimateMaps, refinedMaps

In [11]:
import h5py

hazyImages, clearImages, estimateMaps, refinedMaps = GenerateTransmissionMaps(datasetType=DatasetType.Train)

save_path = "dehaze.hdf5"

hf = h5py.File(save_path, "a")
dset = hf.create_dataset("hazy_image", data=hazyImages)
dset = hf.create_dataset("clear_image", data=clearImages)
dset = hf.create_dataset("transmission_map_refined", data=refinedMaps)
hf.close()  # close the hdf5 file
print("hdf5 file size: %d bytes" % os.path.getsize(save_path))

960 960
SS594_Multispectral_Dehazing/Haze1k/Haze1k/Haze1k_moderate/dataset/train/input/265.png SS594_Multispectral_Dehazing/Haze1k/Haze1k/Haze1k_moderate/dataset/train/target/265.png
SS594_Multispectral_Dehazing/Haze1k/Haze1k/Haze1k_moderate/dataset/train/input/271.png SS594_Multispectral_Dehazing/Haze1k/Haze1k/Haze1k_moderate/dataset/train/target/271.png
956 956 956 956
hdf5 file size: 3508537344 bytes


In [12]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}